In [32]:
import os
import librosa
import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle 
import json
from io import BytesIO


file_path = '/users/seanmullins333/desktop/Heart.mp3'


final_json = {
            'MFCCs': [],
            'Spec_Con': [] }

with open('mlp_model2.pkl', 'rb') as file:
    model = pickle.load(file)
        
with open('SVM-scaler', 'rb') as file:
    scaler = pickle.load(file)
    
with open('SVM-key', 'r') as file:
    SVM_key = json.load(file)

n_mfcc = 40

y, sr = librosa.load(file_path, sr=None, offset=30, duration=100)

mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)

sc = librosa.feature.spectral_contrast(y=y, sr=sr)  

final_json['MFCCs'].append(mfccs)
final_json['Spec_Con'].append(sc)

mfcc_df_mean = pd.DataFrame(np.mean(final_json['MFCCs'][0], axis=1)).transpose()
mfcc_df_var = pd.DataFrame(np.var(final_json['MFCCs'][0], axis=1)).transpose()

sc_df = pd.DataFrame([np.mean(inner_array, axis=1) for inner_array in final_json['Spec_Con']])
mfcc_df = mfcc_df_mean.merge(mfcc_df_var, how='left', left_index=True, right_index=True)
mfcc_df = mfcc_df.merge(sc_df, how='left', left_index=True, right_index=True)
new_column_names = ['Column' + str(i+1) for i in range(len(mfcc_df.columns))]
mfcc_df.columns = new_column_names

mfcc_df_scaled = scaler.transform(mfcc_df)

y_pred = model.predict(mfcc_df_scaled)
y_proba = model.predict_proba(mfcc_df_scaled)

max_proba = round(y_proba.max() * 100, 2)

keys = [key for key, value in SVM_key.items() if value == y_pred[0]]

if keys is None:
    keys = ''

else:
    pass    



In [33]:
pd.DataFrame(y_proba)

,0,1,2,3,4,5,6,7,8
0,0.041956,0.046316,0.026147,0.008214,0.0028,0.014203,0.025593,0.150241,0.68453
